In [1]:
import glob
import matplotlib.pyplot as plt
import pandas as pd
from numpy.fft import fft, ifft
import numpy as np
import scipy
import seaborn as sns
import re

#IMPORTING FUNCTIONS FOR CLASSIFIERS
from sklearn.model_selection import cross_val_score, train_test_split, KFold, StratifiedShuffleSplit, TimeSeriesSplit, RepeatedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier

#IMPORTING FUNCTIONS FOR CLASSIFIERS METRICS
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score

#OPTMIZE PARAMETHERS
import optuna

#IMPORTING FUNCTIONS FROM ./defs

from defs import defs_

In [2]:
#PATHS USED
path_quick_test = [
    "./database_raw/DatasetA/zzzAD1.txt",
    "./database_raw/DatasetA/zzzAD30.txt",
    "./database_raw/DatasetA/zzzAD12.txt",
]

path = glob.glob("./database_raw/DatasetA/zzzA*.txt", recursive=True)


In [3]:
## Create pratical database with three different scenarios available on DATASETA, use to test code before apply on full database in order to save time
# df_quick_test = defs_.prepareDatabase(path_quick_test);
### Instead of creating database every time, it was exported as a csv file named 'dataframesimples' on root of project, imported using following code
# df_final.to_csv('dataframe_simples')

## Create full database with all data available on DATASETA
# df_final = defs_.prepareDatabase(path);
### Instead of creating database every time, it was exported as a csv file named 'dataframe_final' on root of project, imported using following code
# df_final.to_csv('dataframe_final')


# df_final

In [4]:
# Initiate DF's 

## Create full database with all data available on DATASETA
# df_final = defs_.prepareDatabase(path);
### Instead of creating database every time, it was exported as a csv file named 'dataframe_final' on root of project, imported using following code
#######
df_final = pd.read_csv('dataframe_final', dtype='unicode');
#######

## Create pratical database with three different scenarios available on DATASETA, use to test code before apply on full database in order to save time
# df_quick_test = defs_.prepareDatabase(path_quick_test);
### Instead of creating database every time, it was exported as a csv file named 'dataframesimples' on root of project, imported using following code
#######
df_quick_test = pd.read_csv('dataframe_simples', dtype='unicode');
#######

In [6]:
#######
## Create dataframe with separated sensors on quick pratical approach
#######
df_new_final = defs_.createDatabaseSingleSensor(df_final,1)

#######
## Create dataframe with separated sensors on quick pratical approach
#######
df_new_quick = defs_.createDatabaseSingleSensor(df_quick_test,1)

df_new_quick

c:\Users\User\Documents\StructuralHealthMonitoring-TCC\defs\defs_.py:175: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['Scenario'] = df_filtered['Scenario'].apply(transform)
c:\Users\User\Documents\StructuralHealthMonitoring-TCC\defs\defs_.py:175: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['Scenario'] = df_filtered['Scenario'].apply(transform)


,mean_S1,std_S1,median_S1,skew_S1,amp_max_min_S1,kurtosis_S1,damaged
0,-0.000414370442,0.043120406327645536,-0.003243883,-0.41442675239991716,0.18522184,0.18229650179968981,1
1,-0.00149953008,0.08873689907904413,-0.0073719135000000005,-0.008557772829157796,0.3630676,-0.6650972729370102,1
2,-0.002099744577200001,0.12369873116059492,-0.01570684,0.588571492652289,0.6131021999999999,0.7530364721892133,1
3,-0.0033677626000000005,0.1388221230414223,-0.042125805,0.26839345162009703,0.5585321,-0.7062843969531509,1
4,0.0033172219000000003,0.13300805870201626,-0.0047542875,0.3868517885124701,0.5399411000000001,-0.14875481556053805,1
...,...,...,...,...,...,...,...
15724,0.004055566004,0.17509878648950966,0.012669401,0.1482822557076639,0.7799347999999999,-0.08531892569890953,0
15725,-0.018904614004,0.16351748460468576,-0.0049219428499999995,-0.17872425996967795,0.6363976,-0.8093398351879841,0
15726,0.006331191380000001,0.16095294831202925,0.017822967,-0.11571819701910263,0.7339705999999999,-0.30775402848181344,0
15727,-0.016735006046,0.1692349426031033,-0.016996605,0.3648674746155068,0.8232618,0.2717300461434293,0


In [7]:
# Start random forest Model

df_rf = df_new_final

# Setting target variable
y = df_rf['damaged']

# Setting other variables
X = df_rf.drop(columns=['damaged'])

X = X.astype(float)

# Start random forest Model for quick database

df_rf_quick = df_new_quick

# Setting target variable
y_quick = df_rf_quick['damaged']

# Setting other variables
X_quick = df_rf_quick.drop(columns=['damaged'])

X_quick = X_quick.astype(float)


In [8]:
# Segregate database for test and train

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0, shuffle=True) 

std_scaler = StandardScaler()
X_scaled_train = pd.DataFrame(std_scaler.fit_transform(X_train), columns=X_train.columns, index=X_train.index)
X_scaled_test = pd.DataFrame(std_scaler.transform(X_test),columns=X_train.columns,index=X_test.index)

# Segregate database for test and train around the quick test database

X_train_quick, X_test_quick, y_train_quick, y_test_quick = train_test_split(X_quick, y_quick, test_size=0.3, random_state=0, shuffle=True) 
X_scaled_train_quick = pd.DataFrame(std_scaler.fit_transform(X_train_quick), columns=X_train_quick.columns, index=X_train_quick.index)
X_scaled_test_quick = pd.DataFrame(std_scaler.transform(X_test_quick),columns=X_train_quick.columns,index=X_test_quick.index)

In [14]:
### DEFAULT PARAMETHERS ON FULL DATAFRAME 0.9682116488925349

rf = RandomForestClassifier(n_estimators=100)

rf.fit(X_scaled_train,y_train)
y_pred = rf.predict(X_scaled_test)

accuracy_ = accuracy_score(y_test, y_pred)
accuracy_

0.9682116488925349

In [15]:
### OVERFITTING QUICK TEST FOR DEFAULT PARAMETHERS ON FULL DATAFRAME 0.9999033162525379

rf = RandomForestClassifier(n_estimators=100)

rf.fit(X_scaled_train,y_train)
y_pred = rf.predict(X_scaled_train)

accuracy_ = accuracy_score(y_train, y_pred)
accuracy_

0.9999033162525379

In [19]:
### OPTMAZING PARAMETHERS TO MAXIMIZE SCORE ON FULL DATAFRAME

def objective(trial):
    ### PARAMS
    param_n_estimators = trial.suggest_int('n_estimators',100,2000, step = 50);
    # param_max_features = trial.suggest_int('max_features',2,40);
    param_max_depth = trial.suggest_int('max_depth',1,50);


    rf = RandomForestClassifier(n_estimators=param_n_estimators, max_samples= 0.4 , max_depth = param_max_depth )

    rf.fit(X_scaled_train,y_train)
    y_pred = rf.predict(X_scaled_test)
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

study.best_params

[I 2023-05-08 06:00:38,551] A new study created in memory with name: no-name-16da47bf-b300-46ce-9af8-8c891ba0405c
[I 2023-05-08 06:01:09,956] Trial 0 finished with value: 0.9682116488925349 and parameters: {'n_estimators': 400, 'max_depth': 11}. Best is trial 0 with value: 0.9682116488925349.
[I 2023-05-08 06:02:11,187] Trial 1 finished with value: 0.9682116488925349 and parameters: {'n_estimators': 500, 'max_depth': 30}. Best is trial 0 with value: 0.9682116488925349.
[I 2023-05-08 06:05:36,482] Trial 2 finished with value: 0.9682116488925349 and parameters: {'n_estimators': 1700, 'max_depth': 26}. Best is trial 0 with value: 0.9682116488925349.
[I 2023-05-08 06:06:08,399] Trial 3 finished with value: 0.9682116488925349 and parameters: {'n_estimators': 250, 'max_depth': 50}. Best is trial 0 with value: 0.9682116488925349.
[I 2023-05-08 06:07:55,926] Trial 4 finished with value: 0.9682116488925349 and parameters: {'n_estimators': 850, 'max_depth': 45}. Best is trial 0 with value: 0.968

{'n_estimators': 400, 'max_depth': 11}

In [21]:
### OPTMIZED PARAMETHERS OF FULL DATABASE ON FULL DATAFRAME

rf = RandomForestClassifier(n_estimators=100)

rf.fit(X_scaled_train,y_train)
y_pred = rf.predict(X_scaled_test)
accuracy = accuracy_score(y_test, y_pred)

print(accuracy)
rf.feature_importances_

0.9682116488925349


array([0.16355023, 0.17117964, 0.16500328, 0.16482028, 0.16908727,
       0.1663593 ])

In [22]:
optuna.visualization.plot_optimization_history(study)